In [21]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import math
import hashlib
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn import svm,linear_model
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel,RFE,RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler,MinMaxScaler,KBinsDiscretizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR,LinearSVR
from sklearn import model_selection
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from numpy import transpose
from collections.abc import Iterable
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
import os


In [3]:
cwd = os.getcwd()
print(cwd)

/Users/jessicapan/Documents/GitHub/TRPM8-bootcamp-project/4_feature_selection


In [14]:
#Put in fname the csv file of the molecular descriptors calculated for each molecule

fname = "../3_train_test_split/train_set.csv"
df = pd.read_csv("../3_train_test_split/train_set.csv")

def imputing_missing_values(data):
    return SimpleImputer(missing_values=np.NaN, strategy='mean')

imputing_missing_values(fname)
column_names = df.columns
column_names

Index(['PMI1', 'PMI2', 'PMI3', 'Asphericity', 'Eccentricity',
       'InertialShapeFactor', 'NPR1', 'NPR2', 'PBF', 'RadiusOfGyration',
       ...
       'HDonors', 'heteroatoms', 'rotatableBonds', 'saturatedCarbocycles',
       'saturatedHeterocycles', 'satureatedRings', 'ringCount', 'molLogP',
       'molMR', '-logIC50'],
      dtype='object', length=1010)

In [10]:
def write_ft(titles,features):
    with open('ft_selected','w') as f:
        for ft in features:
            f.write(titles[ft]+'\n')

In [11]:
#returns the np.array representation of file 'fname'

target = len(df.columns) - 1
def read_data(fname,target,excluded=[]):
    classification,titles,data = [],[],[]
    
    with open (fname, 'r') as ds:
        titles=ds.readline().strip().split(',')
        target=len(titles)+target if target<0 else target
        titles=[titles[i] for i in range(len(titles)) if i not in excluded and i != target]
        for x in ds:
            line=x.strip().split(',')
            classification.append(float(line[target]))
            data.append([float(line[i]) if len(line[i])>0 else np.NaN for i in range(len(line)) if i not in excluded and i != target ]) 
    return np.array(data),np.array(classification),titles

read_data(fname,target,excluded=[])

(array([[-1.54625991, -0.74380621, -0.94007868, ..., -0.78763571,
         -1.88487169, -1.51316101],
        [ 0.94371803,  0.1903661 ,  0.15864667, ...,  1.31916901,
          1.50430227,  1.67163265],
        [-0.01447707, -1.03467341, -1.01412805, ..., -0.78763571,
         -0.14259884, -0.05613268],
        ...,
        [-0.68147346,  2.28875998,  2.00708531, ...,  0.26576665,
         -0.7447261 ,  0.25848168],
        [-0.59687668, -0.30946505, -0.52721211, ...,  0.26576665,
          0.39479858, -0.32576552],
        [-0.86901284, -0.41552542, -0.41963641, ..., -1.84103807,
         -0.71950164, -1.02012076]]),
 array([13.79570793, 15.96967565, 16.11809565, 15.81061095, 16.15891765,
        18.32537056, 15.84806851, 16.89462444, 16.77202212, 11.29781409,
        16.17997105, 20.19263759, 18.01521564, 15.14052169, 17.16791778,
         9.21034037, 15.35728982, 14.81246919, 15.76743878, 18.89871654,
        17.11234792, 14.71691268, 10.97643209, 12.02375109, 11.51292546,
        

In [16]:
X = df.drop(columns=['-logIC50'])  # Features
y = df['-logIC50']                 # Target variable

In [17]:
def feature_selection_RFECV(X,y,n_tree):
    sel = RandomForestClassifier(n_estimators=n_tree, random_state=40)
    rfecv = RFECV(estimator=sel, step=1,cv=StratifiedKFold(5),   scoring='accuracy')

    selector = rfecv.fit(X,y)
    

    print("\tOptimal number of features : %d" % selector.n_features_)
    # Plot number of features VS. cross-validation scores
    #plt.figure()
    #plt.xlabel("Number of features selected")
    #plt.ylabel("Cross validation score")
    #plt.plot(range(1, len(selector.grid_scores_) + 1), selector.grid_scores_)
    #plt.show()
    return [i for i in range(len(selector.support_)) if selector.support_[i]==True]

feature_selection_RFECV(X,y,n_tree=100)

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
rfecv = RFECV(
    estimator=model, 
    step=1, 
    cv=KFold(5),  # 5-fold cross-validation
    scoring='neg_mean_squared_error'  # Use neg_mean_squared_error for regression
)

# Step 3: Fit the RFECV to the training data
rfecv.fit(X, y)

# Display the optimal number of features
print("Optimal number of features:", rfecv.n_features_)
